# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS event_data 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# Set KEYSPACE to the keyspace specified above


try:
    session.set_keyspace('event_data')
except Exception as e:
    print(e)

### Query 1

#### Create table song_session for query 1

In [10]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

try:
    session.execute(
        "CREATE TABLE IF NOT EXISTS song_session \
        (sessionId int, itemInSession int, artist varchar, songTitle varchar, length float, \
        primary key (sessionId, itemInSession)) \
        "
    )
except Exception as e:
    print(e)
               
# Table song_session Description:   
# In creating this table, I combined sessionId and itemInSession as the partition key.

#### Insert data into table song_session

In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT into song_session (sessionId, itemInSession, artist, songTitle, length)"
        query = query + "values(%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into song_session table

In [12]:
## Add in the SELECT statement to verify the data was entered into the table
try:
    queryout = session.execute('SELECT artist, songTitle, length FROM song_session WHERE sessionId=338 AND itemInSession=4')
except Exception as e:
    print(e)

    
# use pretty table to display data in tabular form and include headings
t = PrettyTable(['Artist', 'Song', 'Length'])
for row in queryout:
    t.add_row([row[0], row[1], row[2]])
print(t)

+-----------+---------------------------------+--------------------+
|   Artist  |               Song              |       Length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


### Query 2

#### Create table user_session for query 2

In [13]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

try:
    session.execute("CREATE TABLE IF NOT EXISTS user_session \
    (userId int, sessionId int, itemInSession int, artist varchar, songTitle varchar, firstName varchar, lastName varchar, \
    PRIMARY KEY ((userId, sessionId), itemInSession))")
except Exception as e:
    print(e)
    
# Table user_session Description: 
# In this query, I used userId combining sessionId as the partition key and itemInSession as my clustering key. 
# Each partition is uniquely identified by userId + sessionId, \
# while itemInSession was used to uniquely identify the rows within a partition to sort the data by the value.

#### Insert data into table user_session

In [14]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT into user_session (userId, sessionId, itemInSession, artist, songTitle, firstName, lastName)"
        query = query + "values(%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into user_session table

In [15]:
## Add in the SELECT statement to verify the data was entered into the table
try:
    queryout = session.execute('SELECT artist, songTitle, firstName, lastName FROM user_session WHERE userId=10 and sessionId=182')
except Exception as e:
    print(e)

# use pretty table to display data in tabular form and include headings
t = PrettyTable(['Artist', 'Song', 'First Name', 'Last Name'])
for row in queryout:
    t.add_row([row[0], row[1], row[2], row[3]])
print(t)

+-------------------+------------------------------------------------------+------------+-----------+
|       Artist      |                         Song                         | First Name | Last Name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


### Query 3

#### Create table user_song for query 3

In [16]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

try:
    queryout = session.execute('\
    CREATE TABLE IF NOT EXISTS user_song \
    (songTitle varchar, userId int, firstName varchar, lastName varchar, \
    PRIMARY KEY (songTitle, userId))')
except Exeption as e:
    print(e)

# Table user_song Description: 
# In this query, I used songTitle combining userId as the partition key.

#### Insert data into table user_song

In [17]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT into user_song (songTitle, userId, firstName, lastName)"
        query = query + "values(%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into user_song table

In [18]:
## Add in the SELECT statement to verify the data was entered into the table
try:
    queryout = session.execute("SELECT firstName, lastName FROM user_song WHERE songTitle='All Hands Against His Own'")
except Exception as e:
    print(e)

# use pretty table to display data in tabular form and include headings
t = PrettyTable(['First Name', 'Last Name'])
for row in queryout:
    t.add_row([row[0], row[1]])
print(t)

+------------+-----------+
| First Name | Last Name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [ ]:
## Drop the table before closing out the sessions

In [ ]:
try:
    queryout = session.execute("DROP TABLE song_session")
except Exception as e:
    print(e)
    
try:
    queryout = session.execute("DROP TABLE user_session")
except Exception as e:
    print(e)
    
try:
    queryout = session.execute("DROP TABLE user_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()